# Guideline for Data Collection
---
- This notebook consists of two parts, including
    * example function for each collection at the begining (5 entries in dataframe format)
    * full program for each collection under different markdown title (from collecting to saving into csv)
    * simple description for each program
- downloaded files are provided

# Collection examples
* web scraping for FIPS code seperately
    - states code
    - county code
* API accessing for PM 2.5 data
    - data frame made in step 1 is needed, please run cell by cell
* No example function for downloadable dataset as they are directly downloaded from CDC platforms, bu a function for showing the 5 entries of it is available.

## example_FIPScode

In [31]:
## FIPS code for states

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def example_statecode():
    # get access to the state webpage
    states_link = 'https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code'
    response = requests.get(states_link)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find and get the state table
    table_content = soup.find('table')

    # headers
    headers = table_content.find('tr')
    if headers:
        column_names = [header.text.strip() for header in headers.find_all('th')] # header text
    column_names.append('county_link')

    # data
    content = table_content.find_all('tr')
    data = []
    for row in content[1:6]:
        raw_data = []
        cells = row.find_all('td')
        if cells:
            raw_data = [cell.text.strip() for cell in cells]
            if row.select_one('table tr td:nth-of-type(4) a'):
                link = row.select_one('table tr td:nth-of-type(4) a')['href']
                if 'wiki' in link:
                    raw_data.append(link)
        data.append(raw_data)

    # convert to pandas dataframe
    df = pd.DataFrame(data, columns = column_names)
    return df

# open test set
example_state = example_FIPScode()
example_state

,Name,Alpha code,Numeric code,Status,county_link
0,Alabama,AL,01,State; counties,/wiki/List_of_counties_in_Alabama
1,Alaska,AK,02,State; boroughs,/wiki/List_of_boroughs_and_census_areas_in_Alaska
2,American Samoa,AS,60,Outlying area under U.S. sovereignty,None
3,American Samoa *,,03,(FIPS 5-1 reserved code),None
4,Arizona,AZ,04,State; counties,/wiki/List_of_counties_in_Arizona


In [2]:
## FIPS code for counties

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def example_countycode(df):
    # county code
    links = df['county_link']
    data_county = []
    for link in links[:5]:
        if link != None:
            response_county = requests.get(f"https://en.wikipedia.org{link}")
            time.sleep(2)
            soup_county = BeautifulSoup(response_county.content, 'html.parser')
            try:
                table_county = soup_county.find('table', {'class': 'wikitable sortable'}) # two different table class name
                content_county = table_county.find_all('tr')
                for county in content_county[1:6]:
                    row_data = []
                    county_name = county.find('th').text.strip()
                    if county.find('td'): # under different tag name
                        FIPS_code = county.find('td').text.strip()
                    else:
                        FIPS_code = county.find('th').text.strip()
                    row_data.append(link)
                    row_data.append(county_name)
                    row_data.append(FIPS_code)
                    data_county.append(row_data)
            except:
                table_county = soup_county.find('table', {'class': 'wikitable sortable plainrowheaders'})
                content_county = table_county.find_all('tr')
                for county in content_county[1:6]:
                    row_data = []
                    county_name = county.find('th').text.strip()
                    if county.find('td'):
                        FIPS_code = county.find('td').text.strip()
                    else:
                        FIPS_code = county.find('th').text.strip()
                    row_data.append(link)
                    row_data.append(county_name)
                    row_data.append(FIPS_code)
                    data_county.append(row_data)

    # convert to pandas dataframe
    data_county = pd.DataFrame(data_county)
    data_county.columns = ['county_link', 'County Name', 'county FIPS Code']

    return data_county

# open test set: some links are missing and would be skipped
df = example_statecode()
example_county = example_countycode(df)
example_county

,county_link,County Name,county FIPS Code
0,/wiki/List_of_counties_in_Alabama,Autauga County,001
1,/wiki/List_of_counties_in_Alabama,Baldwin County,003
2,/wiki/List_of_counties_in_Alabama,Barbour County,005
3,/wiki/List_of_counties_in_Alabama,Bibb County,007
4,/wiki/List_of_counties_in_Alabama,Blount County,009
5,/wiki/List_of_boroughs_and_census_areas_in_Alaska,Aleutians East Borough,013
6,/wiki/List_of_boroughs_and_census_areas_in_Alaska,Anchorage,020
7,/wiki/List_of_boroughs_and_census_areas_in_Alaska,Bristol Bay Borough,060
8,/wiki/List_of_boroughs_and_census_areas_in_Alaska,Denali Borough,068
9,/wiki/List_of_boroughs_and_census_areas_in_Alaska,Fairbanks North Star Borough,090


In [3]:
# pre-processing for FIPS code dataset

# merge two dataframe together based on county link
example_FIPS = pd.merge(example_state, example_county, on = 'county_link')

# drop columns
example_states_counties = example_FIPS.drop(columns = ['Status', 'county_link'])

# rename
example_states_counties.columns = ['State Name', 'Alpha Code', 'State FIPS Code', 'County Name', 'County FIPS Code']

example_states_counties.to_csv('example_states_counties.csv')
example_states_counties

,State Name,Alpha Code,State FIPS Code,County Name,County FIPS Code
0,Alabama,AL,01,Autauga County,001
1,Alabama,AL,01,Baldwin County,003
2,Alabama,AL,01,Barbour County,005
3,Alabama,AL,01,Bibb County,007
4,Alabama,AL,01,Blount County,009
5,Alaska,AK,02,Aleutians East Borough,013
6,Alaska,AK,02,Anchorage,020
7,Alaska,AK,02,Bristol Bay Borough,060
8,Alaska,AK,02,Denali Borough,068
9,Alaska,AK,02,Fairbanks North Star Borough,090


## example_PMdata

In [45]:
# PM 2.5 dataset

import requests
import time
import pandas as pd


def example_PMdata(email, key, years, pollutant, df):
    # make an empty list to store output
    annualdata = []

    # group by state code to make sure looping counties within the same state
    grouped = df.groupby('State FIPS Code')


    # first loop by states and counties
    for state, state_df in grouped:
        # loop by county
        for county in state_df['County FIPS Code'][:2]:
            for year in years:
                bdate = f'{year}0101'
                edate = f'{year}1231'
                # query API
                url = f'https://aqs.epa.gov/data/api/annualData/byCounty?email={email}&key={key}&param={pollutant}&bdate={bdate}&edate={edate}&state={state}&county={county}'
                response = requests.get(url)
                time.sleep(2)
                content = response.json()
                if len(content['Data']) != 0:
                    data = content['Data'][0]
                    first_max = data['first_max_value']
                    second_max = data['second_max_value']
                    third_max = data['third_max_value']
                    fourth_max = data['fourth_max_value']
                    annualdata.append([state, county, year, first_max,second_max, third_max, fourth_max])
                else:
                    first_max = pd.NaT
                    second_max = pd.NaT
                    third_max = pd.NaT
                    fourth_max = pd.NaT
                    annualdata.append([state, county, year, first_max,second_max, third_max, fourth_max])
                    print(f"No data found for state {state} and county {county} in {year}")

    data_pd = pd.DataFrame(annualdata)
    data_pd.columns = ['state_code', 'county_code', 'year', 'first_max','second_max', 'third_max', 'fourth_max']
    return data_pd

example_pmdata = example_PMdata(email = 'siyingya@usc.edu', key = 'ecrukit42', years = ['2011','2012'], pollutant = '88101', df = example_states_counties)
example_pmdata.to_csv('example_pmdata.csv')
example_pmdata

No data found for state 01 and county 001 in 2011
No data found for state 01 and county 001 in 2012
No data found for state 02 and county 013 in 2011
No data found for state 02 and county 013 in 2012


,state_code,county_code,year,first_max,second_max,third_max,fourth_max
0,01,001,2011,NaT,NaT,NaT,NaT
1,01,001,2012,NaT,NaT,NaT,NaT
2,01,003,2011,21.2,21.2,20.3,19.5
3,01,003,2012,24.1,22.0,18.4,17.1
4,02,013,2011,NaT,NaT,NaT,NaT
5,02,013,2012,NaT,NaT,NaT,NaT
6,02,020,2011,195.0,51.0,50.0,49.0
7,02,020,2012,73.0,72.0,72.0,69.0
8,04,001,2011,11.8,7.9,6.2,5.8
9,04,001,2012,7.3,6.5,6.0,5.8


## example_asthma

In [6]:
# asthma dataset
import pandas as pd

def asthma(df):
    example_asthma = pd.read_csv(df)
    return example_asthma[:5]

example_asthma = asthma(df = 'asthma_all.csv')
example_asthma.to_csv('example_asthmadata.csv')
example_asthma

,Unnamed: 0,State FIPS Code,State Name,County FIPS Code,County Name,Year,EDvisit
0,0,19,Iowa,1,Adair,2011,17
1,1,19,Iowa,1,Adair,2012,22
2,2,19,Iowa,1,Adair,2013,21
3,3,19,Iowa,1,Adair,2014,18
4,4,19,Iowa,1,Adair,2015,22


# Guideline for programs building datasets
---
* FIPS code (scraping)
    - states
    - counties
    - combination
* PM 2.5 (API)
* asthma (direct download and append into one new file)

## FIPScode
* scrape State FIPS code and links
* scrape County FIPS code by links
* combine two data frames together based on links

In [35]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Scrap FIPS code for states

def statecode():
    # get access to the state webpage
    states_link = 'https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code'
    response = requests.get(states_link)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find and get the state table
    table_content = soup.find('table')

    # headers
    headers = table_content.find('tr')
    if headers:
        column_names = [header.text.strip() for header in headers.find_all('th')] # header text
    column_names.append('county_link')

    # data
    content = table_content.find_all('tr')
    data = []
    for row in content[1:]:
        raw_data = []
        cells = row.find_all('td')
        if cells:
            raw_data = [cell.text.strip() for cell in cells]
            if row.select_one('table tr td:nth-of-type(4) a'):
                link = row.select_one('table tr td:nth-of-type(4) a')['href']
                if 'wiki' in link:
                    raw_data.append(link)
        data.append(raw_data)
        
    # convert to pandas dataframe
    data_state = pd.DataFrame(data, columns = column_names)
    return data_state

data_state = statecode()

In [36]:
data_state

,Name,Alpha code,Numeric code,Status,county_link
0,Alabama,AL,01,State; counties,/wiki/List_of_counties_in_Alabama
1,Alaska,AK,02,State; boroughs,/wiki/List_of_boroughs_and_census_areas_in_Alaska
2,American Samoa,AS,60,Outlying area under U.S. sovereignty,None
3,American Samoa *,,03,(FIPS 5-1 reserved code),None
4,Arizona,AZ,04,State; counties,/wiki/List_of_counties_in_Arizona
...,...,...,...,...,...
69,Wake Island,WI,79,Minor outlying island territory,None
70,Washington,WA,53,State; counties,/wiki/List_of_counties_in_Washington
71,West Virginia,WV,54,State; counties,/wiki/List_of_counties_in_West_Virginia
72,Wisconsin,WI,55,State; counties,/wiki/List_of_counties_in_Wisconsin


In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# county code
def countycode(links):
    data_county = []
    for link in links:
        if link != None:
            response_county = requests.get(f"https://en.wikipedia.org{link}")
            time.sleep(2)
            soup_county = BeautifulSoup(response_county.content, 'html.parser')
            try:
                table_county = soup_county.find('table', {'class': 'wikitable sortable'}) # two different table class name
                content_county = table_county.find_all('tr')
                for county in content_county[1:]:
                    row_data = []
                    county_name = county.find('th').text.strip()
                    if county.find('td'): # under different tag name
                        FIPS_code = county.find('td').text.strip()
                    else:
                        FIPS_code = county.find('th').text.strip()
                    row_data.append(link)
                    row_data.append(county_name)
                    row_data.append(FIPS_code)
                    data_county.append(row_data)
            except:
                table_county = soup_county.find('table', {'class': 'wikitable sortable plainrowheaders'})
                content_county = table_county.find_all('tr')
                for county in content_county[1:]:
                    row_data = []
                    county_name = county.find('th').text.strip()
                    if county.find('td'):
                        FIPS_code = county.find('td').text.strip()
                    else:
                        FIPS_code = county.find('th').text.strip()
                    row_data.append(link)
                    row_data.append(county_name)
                    row_data.append(FIPS_code)
                    data_county.append(row_data)

    # convert to pandas dataframe
    data_county = pd.DataFrame(data_county)
    data_county.columns = ['county_link', 'County Name', 'county FIPS Code']
    
    return data_county

data_county = countycode(links = data_state['county_link'])

In [ ]:
# merge two dataframe together based on county link
FIPScode = pd.merge(data_state, data_county, on = 'county_link')

# drop columns
FIPS_states_counties = FIPScode.drop(columns = ['Status', 'county_link'])

# rename
FIPS_states_counties.columns = ['State Name', 'Alpha Code', 'State FIPS Code', 'County Name', 'County FIPS Code']

# to csv
FIPS_states_counties.to_csv('states_county_FIPS.csv')

## PMdata
* make a dataframe of available states and merge with the FIPS code data frame
* convert code and state code in query format
* access PM 2.5 data by API
* format correctly

In [16]:
import pandas as pd

# read FIPS code csv
FIPS = pd.read_csv('states_county_FIPS.csv')

# data frame for states that are available for asthma data from 2011 to 2019
states_asthma = pd.DataFrame(['Arizona', 'California', 'Connecticut', 'Florida', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Massachusetts', 'Minnesota', 'Missouri', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'Rhode Island', 'Tennessee', 'Utah', 'Vermont', 'Wisconsin'])
states_asthma.columns = ['State Name']
states_available = pd.merge(FIPS, states_asthma, on = 'State Name')
states_available.drop('Unnamed: 0', axis = 1, inplace = True)
states_available

,State Name,Alpha Code,State FIPS Code,County Name,County FIPS Code
0,Arizona,AZ,4,Apache County,1
1,Arizona,AZ,4,Cochise County,3
2,Arizona,AZ,4,Coconino County,5
3,Arizona,AZ,4,Gila County,7
4,Arizona,AZ,4,Graham County,9
...,...,...,...,...,...
1194,Wisconsin,WI,55,Waukesha County,133
1195,Wisconsin,WI,55,Waupaca County,135
1196,Wisconsin,WI,55,Waushara County,137
1197,Wisconsin,WI,55,Winnebago County,139


In [17]:
import pandas as pd

# format county code
states_available['County FIPS Code'] = states_available['County FIPS Code'].astype(str)
def format_county_code(x):
    if len(str(x)) == 1:
        x = f'00{x}'
    elif len(str(x)) == 2:
        x = f'0{x}'
    else:
        x = x
    return x
states_available.loc[:, 'County FIPS Code'] = states_available['County FIPS Code'].apply(format_county_code)

# format state code
states_available['State FIPS Code'] = states_available['State FIPS Code'].astype(str)
def format_state_code(x):
    if len(str(x)) == 1:
        x = f'0{x}'
    else:
        x = x
    return x
states_available.loc[:, 'State FIPS Code'] = states_available['State FIPS Code'].apply(format_state_code)

In [28]:
import requests
import time
import pandas as pd

# email, key are parameters required for registration
# years is the time range
# pollutant is the parameter for required pollutant
# dataframe is a dataframe contains FIPS code for states and counties

def query_api(email, key, years, pollutant, dataframe):
    
    # make an empty list to store output
    annualdata = []

    # group by state code to make sure looping counties within the same state
    grouped = dataframe.groupby('State FIPS Code')


    # first loop by states and counties
    for state, state_df in grouped:
        # loop by county
        for county in state_df['County FIPS Code']:
            for year in years:
                bdate = f'{year}0101'
                edate = f'{year}1231'
                # query API
                url = f'https://aqs.epa.gov/data/api/annualData/byCounty?email={email}&key={key}&param={pollutant}&bdate={bdate}&edate={edate}&state={state}&county={county}'
                response = requests.get(url)
                time.sleep(2)
                content = response.json()
                if len(content['Data']) != 0:
                    data = content['Data'][0]
                    first_max = data['first_max_value']
                    second_max = data['second_max_value']
                    third_max = data['third_max_value']
                    fourth_max = data['fourth_max_value']
                    annualdata.append([state, county, year, first_max,second_max, third_max, fourth_max])
                else:
                    first_max = pd.NaT
                    second_max = pd.NaT
                    third_max = pd.NaT
                    fourth_max = pd.NaT
                    annualdata.append([state, county, year, first_max,second_max, third_max, fourth_max])
                    # to know progress and unavailable data
                    print(f"No data found for state {state} and county {county} in {year}")

    data_pm = pd.DataFrame(annualdata)
    data_pm.columns = ['state_code', 'county_code', 'year', 'first_max','second_max', 'third_max', 'fourth_max']
    return data_pm

In [ ]:
# parameters
email_address = 'siyingya@usc.edu'
api_key = 'ecrukit42'
years_range = ['2011','2012','2013','2014','2015','2016','2017','2018','2019']
pollutant_num = '88101'

data_pm = query_api(email = email_address, key = api_key, years = years_range, pollutant = pollutant_num, dataframe = states_available)
data_pm.to_csv('pmdata_county.csv')

In [25]:
# pre-process output file to format it correctly and output a final csv file
import pandas as pd
pmdata_file = pd.read_csv('pmdata_county.csv')
pmdata_file.columns
pmdata_file['max_pm2.5'] = pmdata_file[['first_max','second_max', 'third_max', 'fourth_max']].max(axis=1)
pmdata_file.drop(['first_max','second_max', 'third_max', 'fourth_max'], axis = 1, inplace = True)
# drop rows by columns with NaN
pmdata_file.dropna(subset=['max_pm2.5'], inplace=True)
pmdata_file.drop('Unnamed: 0', axis = 1, inplace = True)
pmdata_file.columns = ['State FIPS Code', 'County FIPS Code', 'Year', 'Max_pm2.5']
pmdata_file.to_csv('pmdata_all.csv')
pmdata_file

,State FIPS Code,County FIPS Code,Year,Max_pm2.5
0,4,1,2011,11.8
1,4,1,2012,7.3
3,4,1,2014,7.4
4,4,1,2015,6.5
5,4,1,2016,8.7
...,...,...,...,...
10750,55,133,2015,27.2
10751,55,133,2016,25.9
10752,55,133,2017,196.0
10753,55,133,2018,69.0


## AsthmaData
* download all datasets availabl from 2011 to 2019
* append all datasets into one new dataset for asthma data
* format columns correctly
* rename columns

In [40]:
import pandas as pd
import os

# set the directory for input files
input_dir = './asthma'

# store the individual DataFrames
df_list = []

# iterate over each input file
for filename in os.listdir(input_dir):
    # construct the full path to the input file
    input_path = os.path.join(input_dir, filename)
    
    # read in the input file and limit the coding format for unexpected error
    df_list.append(pd.read_csv(input_path, encoding='utf-8'))
    
# concatenate into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

In [41]:
# check columns
print(combined_df.columns)

Index(['StateFIPS', 'State', 'CountyFIPS', 'County', 'Year', 'Value',
       'Data Comment', 'Unnamed: 7', 'Unnamed: 0'],
      dtype='object')


In [42]:
# drop unnecessary columns
combined_df.drop(['Data Comment', 'Unnamed: 7', 'Unnamed: 0'], axis = 1, inplace = True)

In [43]:
# format columns correctly
combined_df['Year'] = combined_df['Year'].apply(lambda x: int(x))
combined_df['StateFIPS'] = combined_df['StateFIPS'].apply(lambda x: int(x))
combined_df['CountyFIPS'] = combined_df['CountyFIPS'].apply(lambda x: int(x))
combined_df['CountyFIPS'] = combined_df['CountyFIPS'].apply(lambda x: str(x)[-3:])

In [44]:
# rename columns 
combined_df.columns = ['State FIPS Code', 'State Name', 'County FIPS Code', 'County Name', 'Year', 'EDvisit']
combined_df.to_csv('asthma_all.csv')
combined_df

,State FIPS Code,State Name,County FIPS Code,County Name,Year,EDvisit
0,19,Iowa,001,Adair,2011,17
1,19,Iowa,001,Adair,2012,22
2,19,Iowa,001,Adair,2013,21
3,19,Iowa,001,Adair,2014,18
4,19,Iowa,001,Adair,2015,22
...,...,...,...,...,...,...
21637,47,Tennessee,189,Wilson,2015,413
21638,47,Tennessee,189,Wilson,2016,299
21639,47,Tennessee,189,Wilson,2017,314
21640,47,Tennessee,189,Wilson,2018,310


# Dataset Conclusion
* FIPS code: states_county_FIPS.csv
* PM 2.5 data: pmdata_all.csv
* asthma data: asthma_all.csv